In [1]:
# Asthetics
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Basic
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
import os
import random
from tqdm.autonotebook import tqdm
import string
import re
from functools import partial
from pprint import pprint

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
from wordcloud import WordCloud, STOPWORDS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, pickle, time, gc, copy, sys
import yaml
import pandas_profiling as pdp
import warnings
from tqdm import tqdm
import re
import json
from transformers import RobertaTokenizer, RobertaModel
import nltk

tqdm.pandas()   #tqdm is used to show any code running with a progress bar. 
pd.set_option('display.max_columns', 100) # 表示できる列数

In [2]:
!nvidia-smi

Tue Jun 22 13:30:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!ls -la /kaggle/input/download-spacy

total 1208
drwxr-xr-x 8 nobody nogroup      0 Jun 20 05:33 .
drwxr-xr-x 9 root   root      4096 Jun 22 13:30 ..
-rw-r--r-- 1 nobody nogroup  61853 Jun 20 05:33 __notebook__.ipynb
-rw-r--r-- 1 nobody nogroup 835867 Jun 20 05:33 __output__.json
-rw-r--r-- 1 nobody nogroup 325665 Jun 20 05:33 __results__.html
-rw-r--r-- 1 nobody nogroup      0 Jun 20 05:33 custom.css
drwxr-xr-x 2 nobody nogroup      0 Jun 20 05:33 en_core_web_lg-3.0.0-py3-none-any.whl
drwxr-xr-x 2 nobody nogroup      0 Jun 20 05:33 en_core_web_sm-3.0.0-py3-none-any.whl
drwxr-xr-x 2 nobody nogroup      0 Jun 20 05:33 en_core_web_trf-3.0.0-py3-none-any.whl
drwxr-xr-x 2 nobody nogroup      0 Jun 20 05:33 spacy
drwxr-xr-x 2 nobody nogroup      0 Jun 20 05:33 spacy-transformers
drwxr-xr-x 2 nobody nogroup      0 Jun 20 05:33 spacy_cuda110


In [4]:
!pip install -U spacy[cuda110] --no-index --find-links /kaggle/input/download-spacy/spacy_cuda110/ # gpu
# !pip install -U spacy --no-index --find-links /kaggle/input/download-spacy/spacy/ # cpu

Looking in links: /kaggle/input/download-spacy/spacy_cuda110/
Processing /kaggle/input/download-spacy/spacy_cuda110/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/download-spacy/spacy_cuda110/typer-0.3.2-py3-none-any.whl
Processing /kaggle/input/download-spacy/spacy_cuda110/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/download-spacy/spacy_cuda110/catalogue-2.0.4-py3-none-any.whl
Processing /kaggle/input/download-spacy/spacy_cuda110/pydantic-1.7.4-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/download-spacy/spacy_cuda110/thinc-8.0.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/download-spacy/spacy_cuda110/pathy-0.5.2-py3-none-any.whl
Processing /kaggle/input/download-spacy/spacy_cuda110/spacy_legacy-3.0.6-py2.py3-none-any.whl
Processing /kaggle/input/download-spacy/spacy_cuda110/smart_open-3.0.0.tar.gz
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=

In [5]:
!pip install /kaggle/input/download-spacy/en_core_web_lg-3.0.0-py3-none-any.whl/en_core_web_lg-3.0.0-py3-none-any.whl
# roberta
!pip install spacy-transformers --no-index --find-links /kaggle/input/download-spacy/spacy-transformers
!pip install /kaggle/input/download-spacy/en_core_web_trf-3.0.0-py3-none-any.whl/en_core_web_trf-3.0.0-py3-none-any.whl

Processing /kaggle/input/download-spacy/en_core_web_lg-3.0.0-py3-none-any.whl/en_core_web_lg-3.0.0-py3-none-any.whl
  Attempting uninstall: en-core-web-lg
    Found existing installation: en-core-web-lg 2.3.1
    Uninstalling en-core-web-lg-2.3.1:
      Successfully uninstalled en-core-web-lg-2.3.1
Looking in links: /kaggle/input/download-spacy/spacy-transformers
Processing /kaggle/input/download-spacy/spacy-transformers/spacy_transformers-1.0.3-py2.py3-none-any.whl
Processing /kaggle/input/download-spacy/spacy-transformers/spacy_alignments-0.8.3-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/download-spacy/en_core_web_trf-3.0.0-py3-none-any.whl/en_core_web_trf-3.0.0-py3-none-any.whl


In [6]:
import spacy
print(spacy.prefer_gpu())
# nlp = spacy.load('en_core_web_lg') # CPU optimized, not transformer
# nlp = spacy.load('en_core_web_trf') # RoBERTa
# finetuned_ner_nlp = spacy.load("../input/spacy-ner-aug-lexeme/model-best") # lexeme table + data augmentation using the gov data
finetuned_ner_nlp = spacy.load("../input/spacynerlexeme0620/model-best") # added lexeme table

True


In [7]:
def read_append_return(filename, train_files_path, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data
    
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


def calc_score(y_true, y_pred, beta=0.5):
    y_true = y_true.split('|')
    y_pred = y_pred.split('|')
#     print(y_true)
#     print(y_pred)
    TP = 0
    FP = 0
    FN = 0
    for i in range(len(y_pred)):
        FP += 1
        for j in range(len(y_true)):
            if jaccard(y_true[j], y_pred[i])>=0.5:
                FP -= 1
                break
    for i in range(len(y_true)):
        FN += 1
        for j in range(len(y_pred)):
            if jaccard(y_true[i], y_pred[j])>=0.5:
                FN -= 1
                TP += 1
                break
    return TP, FP, FN

def detect_duplicated(x):
    for i in range(len(df_train_reduced)):
        if x==df_train_reduced['text'][i]:
            return df_train_reduced['Id'][i]
    return 'no dup'

def pickle_save(path, df):
    with open(path, 'wb') as f:
        pickle.dump(df, f)

def pickle_load(path):
    with open(path, 'rb') as f:
        df = pickle.load(f)
    return df

def ri(df):
    return df.reset_index(drop=True)


In [8]:
def det_acronym_ver1(text, keywords, TH_LEN_CHAR = 3):
    ans = []
    # text = re.sub("-", " ", text)
    words = text.split()
    for i, word in enumerate(words):
        if word[0]!='(' or word[-1]!=')': continue # (XXX)の形でなければスルー
        acronym_cand = word[1:-1]
        if acronym_cand.lower()==acronym_cand: continue # 大文字が一つもないならスルー
        len_acronym_cand = len(acronym_cand)
        if len_acronym_cand<TH_LEN_CHAR: continue # 3文字以下ならスルー
        acronym_cand_lower = acronym_cand.lower()
        acronym_cand_reverse = acronym_cand_lower[::-1]
        words_cand = words[np.clip(i-len_acronym_cand*3, 0, i):i] # (XXX)の前の数単語を抽出
        words_cand = ' '.join(words_cand).strip()
        words_cand = re.sub('[^A-Za-z0-9]+', ' ', words_cand).strip()
        words_cand = words_cand.split(" ")
        words_cand_reverse = words_cand[::-1]
        idx = 0 # acronym_candの文字index
#         print("words_cand_reverse[0]", words_cand_reverse[0])
        for j, word in enumerate(words_cand_reverse):
            if idx==len_acronym_cand:
                break
            if len(word)==0:
                continue
            if word[0].lower()==acronym_cand_reverse[idx]:
                if idx==len_acronym_cand-1: # 1文字目を検出
                    idx_start = j # dataset名の1単語目のindex
                idx += 1
        if idx==len_acronym_cand:
            words_reverse = words_cand_reverse[:idx_start+1]
            dataset = ' '.join(words_reverse[::-1]).strip().lower()
            if detect_keywords(dataset, keywords):
                acronym = acronym_cand
#                 print("Acronym: {}".format(acronym))
#                 print("Dataset: {}".format(dataset))
                ans.append([acronym_cand, dataset])
        return ans

In [9]:
def chunk_text0(text, CHUNK_SIZE = 512): # textを文に分けkeywordを含むものだけを抽出
    sentences_i = nltk.tokenize.sent_tokenize(text)
    sentences_i2 = []
    for j, sentence in enumerate(sentences_i):
        for keyword in keywords:
            if keyword in sentence.lower():
                sentences_i2.append(sentence)
    token_chunks = []
    for j, sentence in enumerate(sentences_i2):
        token_j = tokenizer(sentence)['input_ids'][1:-1]
        num_chunk = int(np.ceil(len(token_j)/CHUNK_SIZE))
        for k in range(num_chunk):
            token_chunks.append(token_j[k*CHUNK_SIZE:(k+1)*CHUNK_SIZE])
    text_chunks = [tokenizer.decode(chunk) for chunk in token_chunks]
    return text_chunks

def chunk_text(text, CHUNK_SIZE = 512, keywords=[]): # textを文に分けkeywordを含むものだけを抽出
    sentences_i = nltk.tokenize.sent_tokenize(text)
    sentences_i2 = []
    for j, sentence in enumerate(sentences_i):
        for keyword in keywords:
            if keyword in sentence.lower():
                sentences_i2.append(sentence)
    token_chunks = []
    for j, sentence in enumerate(sentences_i2):
        token_j = tokenizer(sentence)['input_ids'][1:-1]
        num_chunk = int(np.ceil(len(token_j)/CHUNK_SIZE))
        for k in range(num_chunk):
            token_chunks.append(token_j[k*CHUNK_SIZE:(k+1)*CHUNK_SIZE])
    text_chunks = [tokenizer.decode(chunk) for chunk in token_chunks]
    text_chunks2 = []
    for j, chunk in enumerate(text_chunks):
        for keyword in keywords:
            if keyword in chunk.lower():
                text_chunks2.append(chunk)
    return text_chunks2

def get_df2(sentence_list):
    df_ner = []
    for i, sent in enumerate(sentence_list):
        doc_tmp = nlp(sent) # sentenceで区切ってバッチ化して処理される
    #     print(doc_tmp)
        df_tmps = [str(chunk) for chunk in doc_tmp.noun_chunks]
        if len(df_tmps)>0:
            df_tmps = pd.DataFrame(df_tmps, columns=['chunk'])
            df_tmps['sentence_idx'] = i
            df_ner.append(df_tmps)
    if len(df_ner)>0:
        df_ner = pd.concat(df_ner).reset_index(drop=True)
    else:
        df_ner = pd.DataFrame(columns=['chunk'])
    return df_ner

def monitor_loop(i, len_data, start, verbose):
    if (i+1)%verbose==0:
        print("{}/{}, sec: {:.1f}".format(i+1, len_data, time.time()-starttime))

In [10]:
def check_no_capital(x):
    try:
        if x[1:]==x[1:].lower():
            return True
        return False
    except:
        return False

def delete_the(x):
    try:
        x_split = x.split()
        if x_split[0]=='the':
            return ' '.join(x_split[1:])
        return x
    except:
        return x

def check_including_acronym(x):
    try:
        x_split = re.split("[ ']" , x)
        for word in x_split:
            if word.upper()==word and word.lower()!=word: # 大文字だけの単語があるか
                return True
        return False
    except:
        return True    
    
def get_match(x, ref_labels, threshold=0.5):
    for label in ref_labels:
        if jaccard(x, label)>=threshold:
            return label
    return 'no_match'

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    if len(a)==0 or len(b)==0:
        return 0
    return float(len(c)) / (len(a) + len(b) - len(c))

def delete_keywords(x, ref):
    try:
        x_split = x.split()
        x_new = []
        for item in x_split:
            if item.lower() not in ref:
                x_new.append(item)
        x_new = ' '.join(x_new)
        return x_new
    except:
        return ''


def check_head_capital(x):
    try:
        if len(x)==0:
            return False
        return x[0]==x[0].upper() and x[0]!=x[0].lower() 
    except:
        return False

def detect_black(x, ref):
    try:
        for item in ref:
            if item in x:
                return True
        return False
    except:
        return True
    
def detect_label(x, ref_label, ref_target):
    try:
        predict = []
        for i in range(len(ref_label)):
            if ref_label[i] in x:
                predict.append(ref_target[i])
        return np.array(predict, np.int64)
    except:
        return np.array([], np.int64)
    
def get_label(x, ref_label, ref_target):
    try:
        predict = []
        for i in range(len(x)):
            predict.append(ref_label[ref_target==x[i]][0])
        predict = np.unique(predict).tolist()
        predict = '|'.join(predict)
        return predict
    except:
        return ''
    
def detect_acronym(x, ref_label, ref_acronym, ref_target, th=[1,2]):
    try:
        ans = []
        for i in range(len(ref_label)):
            if x.count(ref_label[i])>=th[0] and x.count(ref_acronym[i])>=th[1]:
                ans.append(ref_target[i])
    #             print(i)
        return ans
    except:
        return []
    
def detect_keywords(x, ref):
    try:
        for keyword in ref:
            if keyword in x:
                return True
        return False
    except:
        return False

In [11]:
keywords = [
    'study',
    'studies',
    'data',
    'survey',
    'panel',
    'census',
    'cohort',
    'longitudinal',
    'registry',   
]

keywords2 = [
    'study',
    'studies',
    'data',
    'survey',
    'panel',
    'census',
    'cohort',
    'longitudinal',
    'registry',
    'the',
]
keywords3 = [
    'study',
    'studies',
    'dataset',
    'database',
    'survey',
    'panel',
    'census',
    'cohort',
    'longitudinal',
    'registry',
]
keywords4 = [
    'system',
    'center',
    'centre',
    'committee',
    'documentation',
    'entry',
    'assimilation',
    'explorer',
    'regulation',
    'portal',
    'format',
    'data science',
    'analysis',
    'management',
    'agreement',
    'branch',
    'acquisition',
    'request',
    'task force',
    'program',
    'operator',
    'office',
    'data view',
    'data language',
    'mission',
    'alliance',
    'data model',
    'data structure',
    'corporation',
]

white_list = [
    'ipeds',
]

keywords5 = keywords + ['of', 'the', 'national', 'education']

ng_list = [
    'national longitudinal survey',
    'education longitudinal survey',
    'census bureau',
    'data appendix',
    'data file user',
    'supplementary data',
    'data supplement',
    'major field of study'
]
black_list = [
    'USGS',
    'GWAS',
    'ECLS',
    'DAS',
    'NCDC',
    'NDBC',
    'UDS',
    'GTD',
    'ISC',
    'DGP',
    'EDC',
    'FDA',
    'TSE',
    'DEA',
    'CDA',
    'IDB',
    'NGDC',
    'JODC',
    'EDM',
    'FADN',
    'LRD',
    'DBDM',
    'DMC',
    'WSC',
    ###count4##
]

# Data loading

In [12]:
df_train = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
print(df_train.shape)
df_train.head()

(19661, 5)


,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study


In [13]:
df_train_reduced = pd.read_csv("../input/coleridge-ext/df_train_reduced.csv")
print(df_train_reduced.shape)
df_train_reduced.head()

(14316, 7)


,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,clean_text
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...,this study used data from the national educati...
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...,this article investigates an important factor ...


In [14]:
df_test = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")
print(df_test.shape)
df_test.head()

(4, 2)


,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,NaN
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN


In [15]:
test_files_path = "../input/coleridgeinitiative-show-us-the-data/test"
df_test['text'] = df_test['Id'].progress_apply(lambda x: read_append_return(x, train_files_path=test_files_path))
df_test['clean_text'] = df_test['text'].progress_apply(clean_text)
df_test.head()

100%|██████████| 4/4 [00:00<00:00, 148.98it/s]


,Id,PredictionString,text,clean_text
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...,this report describes how the education system...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...,a significant body of research has been conduc...


In [16]:
# find train data in test data
df_test['dup_id'] = df_test['text'].progress_apply(lambda x: detect_duplicated(x))
df_test['dup'] = df_test['dup_id']!='no dup'
if len(df_test)==4:
    df_test['dup'][0] = False
df_test.head()

100%|██████████| 4/4 [00:00<00:00, 15.23it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Id,PredictionString,text,clean_text,dup_id,dup
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...,2100032a-7c33-4bff-97ef-690822c43466,False
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...,this report describes how the education system...,2f392438-e215-4169-bebf-21ac4ff253e1,True
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...,3f316b38-1a24-45a9-8d8c-4e05a42257c6,True
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...,a significant body of research has been conduc...,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,True


In [17]:
df_traintest = pd.concat([df_train_reduced, df_test[df_test['dup']==False]]).reset_index(drop=True)
print(df_traintest.shape)
df_traintest.head()

(14317, 10)


,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,clean_text,PredictionString,dup_id,dup
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...,this study used data from the national educati...,NaN,NaN,NaN
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...,NaN,NaN,NaN
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...,NaN,NaN,NaN
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...,NaN,NaN,NaN
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...,this article investigates an important factor ...,NaN,NaN,NaN


In [18]:
if len(df_test)==4:
    df_test_reduced = df_train_reduced.iloc[:1000][['Id', 'text', 'clean_text']].copy()
    df_test_reduced['text'].iloc[-1] = " "
    df_test_reduced['clean_text'].iloc[-1] = " "
else:
    df_test_reduced = df_test[df_test['dup']==False].reset_index(drop=True).iloc[:][['Id', 'text', 'clean_text']]

print(df_test_reduced.shape)
df_test_reduced.head()

(1000, 3)


,Id,text,clean_text
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,This study used data from the National Educati...,this study used data from the national educati...
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...
4,c754dec7-c5a3-4337-9892-c02158475064,This article investigates an important factor ...,this article investigates an important factor ...


In [19]:
if len(df_test)==4:
    df_traintest = df_train_reduced.iloc[:100].copy()
#     df_traintest['text'].iloc[-1] = ""
#     df_traintest['clean_text'].iloc[-1] = ""
#     df_traintest = df_train_reduced
print(df_traintest.shape)
df_traintest.head()

(100, 7)


,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,clean_text
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...,this study used data from the national educati...
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...,this article investigates an important factor ...


# train label

In [20]:
df_label = df_train['cleaned_label'].value_counts().reset_index()
df_label.columns = ['cleaned_label', 'count']
df_label['-count'] = -df_label['count']
df_label = df_label.sort_values(['-count', 'cleaned_label']).reset_index(drop=True)
df_label['target'] = np.arange(len(df_label))
df_label['cleaned_label'] = df_label['cleaned_label'].apply(clean_text)

df_tmp1 = df_train[['dataset_label', 'cleaned_label', 'dataset_title']]
df_tmp2 = df_train[['dataset_title', 'cleaned_label', 'dataset_title']]
df_tmp3 = df_train[['cleaned_label', 'cleaned_label', 'dataset_title']]
df_tmp1.columns = ['label', 'cleaned_label', 'dataset_title']
df_tmp2.columns = ['label', 'cleaned_label', 'dataset_title']
df_tmp3.columns = ['label', 'cleaned_label', 'dataset_title']
df_label2 = pd.concat([df_tmp1, df_tmp2, df_tmp3])

df_label2['label'] = df_label2['label'].apply(lambda x: x.lower())
df_label2['cleaned_label_my'] = df_label2['label'].apply(clean_text)
df_label2['cleaned_label'] = df_label2['cleaned_label'].apply(clean_text)
df_label2 = df_label2[df_label2['label'].duplicated()==False].reset_index(drop=True)
df_label2 = pd.merge(df_label2, df_label, on='cleaned_label', how='left')
print(df_label2.shape)
print(np.sum(pd.isna(df_label2['target'])))
# print(df_label2[df_label2['target']==1])
df_label2 = df_label2.sort_values(['-count', 'label']).reset_index(drop=True)
df_label2['target_relabeled'] = df_label2['target']
df_label2['target_relabeled'][df_label2['cleaned_label']!=df_label2['cleaned_label_my']] =\
    np.arange(np.sum(df_label2['cleaned_label']!=df_label2['cleaned_label_my'])) + df_label2['target'].max()+1
print("not match: ", np.sum(df_label2['cleaned_label']!=df_label2['cleaned_label_my']))
print("unique cleaned_label_my: ", len(df_label2['cleaned_label_my'].unique()))
print(df_label2['target'].max())
print(df_label2['target_relabeled'].max())
df_label_train = df_label2
df_label_train.head()

(180, 7)
0
not match:  3
unique cleaned_label_my:  133
129
132


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,cleaned_label,dataset_title,cleaned_label_my,count,-count,target,target_relabeled
0,adni,adni,Alzheimer's Disease Neuroimaging Initiative (A...,adni,3673,-3673,0,0
1,alzheimer s disease neuroimaging initiative adni,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1
2,alzheimer's disease neuroimaging initiative (a...,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1
3,trends in international mathematics and scienc...,trends in international mathematics and scienc...,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,1163,-1163,2,2
4,baltimore longitudinal study of aging,baltimore longitudinal study of aging,Baltimore Longitudinal Study of Aging (BLSA),baltimore longitudinal study of aging,1156,-1156,3,3


In [21]:
df_label_train['label-keywords'] = df_label_train['cleaned_label_my'].apply(
    lambda x: delete_keywords(x, keywords5))
df_label_train.head()

,label,cleaned_label,dataset_title,cleaned_label_my,count,-count,target,target_relabeled,label-keywords
0,adni,adni,Alzheimer's Disease Neuroimaging Initiative (A...,adni,3673,-3673,0,0,adni
1,alzheimer s disease neuroimaging initiative adni,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1,alzheimer s disease neuroimaging initiative adni
2,alzheimer's disease neuroimaging initiative (a...,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1,alzheimer s disease neuroimaging initiative adni
3,trends in international mathematics and scienc...,trends in international mathematics and scienc...,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,1163,-1163,2,2,trends in international mathematics and science
4,baltimore longitudinal study of aging,baltimore longitudinal study of aging,Baltimore Longitudinal Study of Aging (BLSA),baltimore longitudinal study of aging,1156,-1156,3,3,baltimore aging


In [22]:
df_train['target'] = df_train['cleaned_label'].progress_apply(lambda x: df_label2['target'][df_label2['cleaned_label']==x.strip()].values[0])
df_tmp = df_train.groupby('Id')['target'].agg(lambda x: list(x)).reset_index()
df_tmp.columns = ['Id', 'targets']
df_tmp['targets'] = df_tmp['targets'].apply(lambda x: np.array([x]).flatten())
df_train_reduced2 = pd.merge(df_train_reduced, df_tmp, on='Id', how='left')
df_train_reduced2.head()

100%|██████████| 19661/19661 [00:03<00:00, 5193.12it/s]


,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,clean_text,targets
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...,this study used data from the national educati...,"[8, 5]"
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...,"[8, 5]"
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...,"[8, 5]"
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...,"[8, 5]"
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...,this article investigates an important factor ...,"[8, 5]"


# train label acronym

In [23]:
acronym_list = [
    ['National Education Longitudinal Study', 'nels'],
#     ['NOAA Tide Gauge', 'nan'],
    ['Sea, Lake, and Overland Surges from Hurricanes', 'slosh'],
    ['Coastal Change Analysis Program', 'c-cap'],
    ['Aging Integrated Database (AGID)', 'agid'],
    ["Alzheimer's Disease Neuroimaging Initiative (ADNI)", 'adni'],
    ['Baltimore Longitudinal Study of Aging (BLSA)', 'blsa'],
    ['Agricultural Resource Management Survey', 'arms'],
    ['Beginning Postsecondary Student', 'bps'],
    ["The National Institute on Aging Genetics of Alzheimer's Disease Data Storage Site (NIAGADS)", 'niagads'],
    ['Common Core of Data', 'ccd'],
#     ['Survey of Industrial Research and Development', 'nan'],
    ['Baccalaureate and Beyond', 'b&b'],
    ['International Best Track Archive for Climate Stewardship', 'IBTrACS'],
    ['National Teacher and Principal Survey', 'ntps'],
    ['Higher Education Research and Development Survey', 'herd'],
    ['Survey of Earned Doctorates', 'sed'],
    ['School Survey on Crime and Safety', 'ssocs'],
    ['World Ocean Database', 'wod'],
    ['Program for the International Assessment of Adult Competencies', 'piaac'],
    ['Early Childhood Longitudinal Study', 'ecls'],
#     ['Survey of Graduate Students and Postdoctorates in Science and Engineering', 'nan'],
    ['Trends in International Mathematics and Science Study', 'timss'],
    ['Education Longitudinal Study', 'els'],
    ['Optimum Interpolation Sea Surface Temperature', 'oisst'],
    ['National Assessment of Education Progress', 'naep'],
    ['High School Longitudinal Study', 'hsls'],
    ['Survey of Doctorate Recipients', 'sdr'],
    ['Rural-Urban Continuum Codes', 'rucc'],
#     ['Survey of Science and Engineering Research Facilities', 'nan'],
#     ['FFRDC Research and Development Survey', 'nan'],
#     ['Survey of State Government Research and Development', 'nan'],
    ['Advanced National Seismic System (ANSS) Comprehensive Catalog (ComCat)', 'comcat'],
#     ['Census of Agriculture', 'nan'],
    ['North American Breeding Bird Survey (BBS)', 'bbs'],
    ['COVID-19 Open Research Dataset (CORD-19)', 'cord-19'],
    ['Complexity Science Hub COVID-19 Control Strategies List (CCCSL)', 'cccsl'],
#     ['Our World in Data COVID-19 dataset', 'nan'],
    ['COVID-19 Precision Medicine Analytics Platform Registry (JH-CROWN)', 'jh-crown'],
    ['Characterizing Health Associated Risks, and Your Baseline Disease In SARS-COV-2 (CHARYBDIS)', 'charybdis'],
#     ['COVID-19 Deaths data', 'nan'],
#     ['SARS-CoV-2 genome sequence', 'nan'],
#     ['COVID-19 Image Data Collection', 'nan'],
    ['RSNA International COVID-19 Open Radiology Database (RICORD)', 'ricord'],
#     ['CAS COVID-19 antiviral candidate compounds dataset', 'nan'],
]
df_label_train_acronym = pd.DataFrame(acronym_list, columns=['dataset_title', 'acronym'])
df_label_train_acronym['acronym_clean'] = df_label_train_acronym['acronym'].apply(clean_text)
df_label_train_acronym['target'] = np.arange(len(df_label_train_acronym))+df_label_train['target'].max()+1
df_label_train_acronym

,dataset_title,acronym,acronym_clean,target
0,National Education Longitudinal Study,nels,nels,130
1,"Sea, Lake, and Overland Surges from Hurricanes",slosh,slosh,131
2,Coastal Change Analysis Program,c-cap,c cap,132
3,Aging Integrated Database (AGID),agid,agid,133
4,Alzheimer's Disease Neuroimaging Initiative (A...,adni,adni,134
5,Baltimore Longitudinal Study of Aging (BLSA),blsa,blsa,135
6,Agricultural Resource Management Survey,arms,arms,136
7,Beginning Postsecondary Student,bps,bps,137
8,The National Institute on Aging Genetics of Al...,niagads,niagads,138
9,Common Core of Data,ccd,ccd,139


In [24]:
TH_LEN_SELF = 4
df_label_train_acronym2 = pd.merge(df_label_train_acronym, df_label_train[['cleaned_label_my', 'dataset_title']], on='dataset_title', how='left')
print(df_label_train_acronym2.shape)
df_label_train_acronym2['base'] = df_label_train_acronym2['cleaned_label_my']
df_label_train_acronym2['label'] = df_label_train_acronym2['acronym_clean']
df_label_train_acronym2['len'] = df_label_train_acronym2['acronym_clean'].apply(lambda x: len(x))
df_label_train_acronym2 = ri(df_label_train_acronym2[df_label_train_acronym2['len']>=TH_LEN_SELF])
print(df_label_train_acronym2.shape)
df_label_train_acronym2.head()

(105, 5)
(81, 8)


,dataset_title,acronym,acronym_clean,target,cleaned_label_my,base,label,len
0,National Education Longitudinal Study,nels,nels,130,national education longitudinal study,national education longitudinal study,nels,4
1,"Sea, Lake, and Overland Surges from Hurricanes",slosh,slosh,131,sea lake and overland surges from hurricanes,sea lake and overland surges from hurricanes,slosh,5
2,"Sea, Lake, and Overland Surges from Hurricanes",slosh,slosh,131,slosh model,slosh model,slosh,5
3,"Sea, Lake, and Overland Surges from Hurricanes",slosh,slosh,131,noaa sea lake and overland surges from hurricanes,noaa sea lake and overland surges from hurricanes,slosh,5
4,"Sea, Lake, and Overland Surges from Hurricanes",slosh,slosh,131,noaa sea lake and overland surges from hurricanes,noaa sea lake and overland surges from hurricanes,slosh,5


In [25]:
df_label_train_ref = df_label_train.copy()
print(df_label_train_ref['dataset_title'].unique().shape)
df_agg = df_label_train_ref.groupby('dataset_title')['target_relabeled'].agg(min).reset_index()
df_agg.columns = ['dataset_title', 'target_group']
df_label_train_ref = pd.merge(df_label_train_ref, df_agg, on='dataset_title', how='left')
df_label_train_ref = df_label_train_ref[df_label_train_ref['dataset_title']!='National Education Longitudinal Study']
df_label_train_ref = df_label_train_ref[df_label_train_ref['dataset_title']!='Education Longitudinal Study']
df_label_train_ref = ri(df_label_train_ref)
print(df_label_train_ref['target_group'].unique().shape)
print(df_label_train_ref.shape)
df_label_train_ref.head()

(45,)
(43,)
(178, 10)


,label,cleaned_label,dataset_title,cleaned_label_my,count,-count,target,target_relabeled,label-keywords,target_group
0,adni,adni,Alzheimer's Disease Neuroimaging Initiative (A...,adni,3673,-3673,0,0,adni,0
1,alzheimer s disease neuroimaging initiative adni,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1,alzheimer s disease neuroimaging initiative adni,0
2,alzheimer's disease neuroimaging initiative (a...,alzheimer s disease neuroimaging initiative adni,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni,2400,-2400,1,1,alzheimer s disease neuroimaging initiative adni,0
3,trends in international mathematics and scienc...,trends in international mathematics and scienc...,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,1163,-1163,2,2,trends in international mathematics and science,2
4,baltimore longitudinal study of aging,baltimore longitudinal study of aging,Baltimore Longitudinal Study of Aging (BLSA),baltimore longitudinal study of aging,1156,-1156,3,3,baltimore aging,3


# Detect Acronym ver1

In [26]:
df_test_reduced['det_acronym'] = df_test_reduced['text'].progress_apply(lambda x: det_acronym_ver1(x, keywords))
df_test_reduced.head()

100%|██████████| 1000/1000 [00:00<00:00, 2355.48it/s]


,Id,text,clean_text,det_acronym
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,This study used data from the National Educati...,this study used data from the national educati...,[]
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...,[]
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...,[]
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...,[]
4,c754dec7-c5a3-4337-9892-c02158475064,This article investigates an important factor ...,this article investigates an important factor ...,"[[NELS, national education longitudinal study]]"


In [27]:
tmp = []
for i in range(len(df_test_reduced)):
    if df_test_reduced['det_acronym'][i] is not None:
        tmp += df_test_reduced['det_acronym'][i]
df_label_acronym_ver1_test = pd.DataFrame(tmp)
df_label_acronym_ver1_test.columns = ['acronym', 'base']
df_label_acronym_ver1_test['base_acronym'] = df_label_acronym_ver1_test['base']+"|"+df_label_acronym_ver1_test['acronym']

df_label_acronym_ver1_test_tmp = df_label_acronym_ver1_test[df_label_acronym_ver1_test['base_acronym'].duplicated()==False]
df_agg = df_label_acronym_ver1_test.groupby('base_acronym')['base'].agg(len).reset_index()
cols_tmp = df_agg.columns.tolist()
cols_tmp[-1] = 'count_BA'
df_agg.columns = cols_tmp
df_label_acronym_ver1_test = pd.merge(df_label_acronym_ver1_test_tmp, df_agg, on='base_acronym', how='left')

print(df_label_acronym_ver1_test.shape)
# BA unique 325
df_label_acronym_ver1_test.head(30)

(48, 4)


,acronym,base,base_acronym,count_BA
0,NELS,national education longitudinal study,national education longitudinal study|NELS,17
1,NLS,national longitudinal study,national longitudinal study|NLS,1
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,1
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,2
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,1
5,NELS,national education longitudinal study of 1988,national education longitudinal study of 1988|...,1
6,NHES,national household education survey,national household education survey|NHES,1
7,NLSY,national longitudinal survey of youth,national longitudinal survey of youth|NLSY,2
8,ATUS,american time use survey,american time use survey|ATUS,1
9,TIMSS,trends in international mathematics and scienc...,trends in international mathematics and scienc...,1


In [28]:
df_label_acronym_ver1_train = pd.read_csv("../input/coleridge-ext/df_label_acronym_ver1_all_210619_02.csv")
print(df_label_acronym_ver1_train.shape)
df_label_acronym_ver1_train.head()

(325, 11)


,acronym,base,base_acronym,count_BA,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym
0,NELS,national education longitudinal study,national education longitudinal study|NELS,24,True,education longitudinal study,national education,2,no_match,nels,True
1,NLS,national longitudinal study,national longitudinal study|NLS,1,True,education longitudinal study,national,1,no_match,nls,False
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,2,True,beginning postsecondary students longitudinal ...,beginning postsecondary student,3,beginning postsecondary student,bps,True
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,2,True,no_match,national educational of 1988,4,no_match,nels,True
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,1,True,no_match,high school and beyond dataset,5,no_match,hsb,False


In [29]:
if len(df_test)==4:
    df_label_acronym_ver1_test['acronym'].iloc[-1] = 'DUMMY'
    df_label_acronym_ver1_test['base'].iloc[-1] = 'dummy'
    df_label_acronym_ver1_test['base_acronym'].iloc[-1] = 'dummy|DUMMY'
df_label_acronym_ver1_test['test'] = True
df_tmp = df_label_acronym_ver1_train.copy()
df_tmp['count_BA_train'] = df_tmp['count_BA']
df_label_acronym_ver1_test_tmp = pd.merge(df_label_acronym_ver1_test, df_tmp[['base_acronym', 'count_BA_train']], on='base_acronym', how='left')
print(df_label_acronym_ver1_test_tmp.shape)
df_label_acronym_ver1_train_tmp = pd.merge(df_label_acronym_ver1_train, df_label_acronym_ver1_test[['base_acronym', 'test']], on='base_acronym', how='left')
df_label_acronym_ver1_train_tmp = ri(df_label_acronym_ver1_train_tmp[df_label_acronym_ver1_train_tmp['test']!=True])
print(df_label_acronym_ver1_train_tmp.shape)
df_label_acronym_ver1 = ri(pd.concat([df_label_acronym_ver1_test_tmp, df_label_acronym_ver1_train_tmp], axis=0))
print(df_label_acronym_ver1.shape)
df_label_acronym_ver1['cleaned_label'] = df_label_acronym_ver1['base'].apply(clean_text)
df_label_acronym_ver1['target'] = np.arange(len(df_label_acronym_ver1))\
    +df_label_train_acronym['target'].max()+1
df_label_acronym_ver1.head()

(48, 6)
(278, 12)
(326, 13)


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,acronym,base,base_acronym,count_BA,test,count_BA_train,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym,cleaned_label,target
0,NELS,national education longitudinal study,national education longitudinal study|NELS,17,True,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,national education longitudinal study,163
1,NLS,national longitudinal study,national longitudinal study|NLS,1,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,national longitudinal study,164
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,1,True,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,beginning postsecondary student longitudinal s...,165
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,2,True,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,national educational longitudinal survey of 1988,166
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,1,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high school and beyond longitudinal dataset,167


In [30]:
df_label_acronym_ver1['match_train'] = df_label_acronym_ver1['base'].apply(
    get_match, ref_labels=df_label_train['cleaned_label_my'])
df_label_acronym_ver1['label-keywords'] = df_label_acronym_ver1['base'].apply(
    lambda x: delete_keywords(x, keywords5))
df_label_acronym_ver1['len_label-keywords'] = df_label_acronym_ver1['label-keywords'].apply(
    lambda x: len(x.split()))
df_label_acronym_ver1['match_train-keywords'] = df_label_acronym_ver1['label-keywords'].apply(lambda x:
    get_match(x, ref_labels=df_label_train['label-keywords'], threshold=0.75))
df_label_acronym_ver1['acronym_clean'] = df_label_acronym_ver1['acronym'].apply(clean_text)
df_label_acronym_ver1['match_train_acronym'] = df_label_acronym_ver1['acronym_clean'].apply(
    lambda x: x in df_label_train_acronym['acronym_clean'].tolist()
)
df_label_acronym_ver1['keyword3'] = df_label_acronym_ver1['base'].apply(lambda x: detect_keywords(x.lower(), keywords3))
df_label_acronym_ver1['keyword4'] = df_label_acronym_ver1['base'].apply(lambda x: detect_keywords(x.lower(), keywords4))
df_label_acronym_ver1['black'] = df_label_acronym_ver1['acronym'].apply(lambda x: x.upper() in black_list)
df_label_acronym_ver1['white'] = df_label_acronym_ver1['acronym_clean'].apply(lambda x: x in white_list)

df_label_acronym_ver1.head()

,acronym,base,base_acronym,count_BA,test,count_BA_train,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym,cleaned_label,target,match_train,keyword3,keyword4,black,white
0,NELS,national education longitudinal study,national education longitudinal study|NELS,17,True,24.0,NaN,NaN,,0,no_match,nels,True,national education longitudinal study,163,education longitudinal study,True,False,False,False
1,NLS,national longitudinal study,national longitudinal study|NLS,1,True,1.0,NaN,NaN,,0,no_match,nls,False,national longitudinal study,164,education longitudinal study,True,False,False,False
2,BPS,beginning postsecondary student longitudinal s...,beginning postsecondary student longitudinal s...,1,True,2.0,NaN,NaN,beginning postsecondary student,3,beginning postsecondary student,bps,True,beginning postsecondary student longitudinal s...,165,beginning postsecondary students longitudinal ...,True,False,False,False
3,NELS,national educational longitudinal survey of 1988,national educational longitudinal survey of 19...,2,True,2.0,NaN,NaN,educational 1988,2,no_match,nels,True,national educational longitudinal survey of 1988,166,no_match,True,False,False,False
4,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,1,True,1.0,NaN,NaN,high school and beyond dataset,5,no_match,hsb,False,high school and beyond longitudinal dataset,167,no_match,True,False,False,False


In [31]:
idx_tmp = (df_label_acronym_ver1['len_label-keywords']>0)
idx_tmp = idx_tmp & (df_label_acronym_ver1['match_train-keywords']=='no_match')
idx_tmp = idx_tmp & (df_label_acronym_ver1['match_train_acronym']==False)
# idx_tmp = idx_tmp & (df_label_acronym_ver1['black']==False)
idx_tmp = idx_tmp & ((df_label_acronym_ver1['keyword3'])|(df_label_acronym_ver1['keyword4']==False))
idx_tmp = idx_tmp | (df_label_acronym_ver1['white'])
idx_tmp = idx_tmp & (df_label_acronym_ver1['cleaned_label'].duplicated()==False)
df_label_acronym_ver1_2 =ri(df_label_acronym_ver1[idx_tmp])
print(df_label_acronym_ver1.shape)
print(df_label_acronym_ver1_2.shape)
print(df_label_acronym_ver1_2[df_label_acronym_ver1_2['cleaned_label'].duplicated()==False].shape)
df_label_acronym_ver1_2.head(30)

(326, 20)
(253, 20)
(253, 20)


,acronym,base,base_acronym,count_BA,test,count_BA_train,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym,cleaned_label,target,match_train,keyword3,keyword4,black,white
0,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,1,True,1.0,NaN,NaN,high school and beyond dataset,5,no_match,hsb,False,high school and beyond longitudinal dataset,167,no_match,True,False,False,False
1,NHES,national household education survey,national household education survey|NHES,1,True,4.0,NaN,NaN,household,1,no_match,nhes,False,national household education survey,169,no_match,True,False,False,False
2,NLSY,national longitudinal survey of youth,national longitudinal survey of youth|NLSY,2,True,4.0,NaN,NaN,youth,1,no_match,nlsy,False,national longitudinal survey of youth,170,no_match,True,False,False,False
3,ATUS,american time use survey,american time use survey|ATUS,1,True,1.0,NaN,NaN,american time use,3,no_match,atus,False,american time use survey,171,no_match,True,False,False,False
4,CPS,current population survey,current population survey|CPS,2,True,13.0,NaN,NaN,current population,2,no_match,cps,False,current population survey,173,no_match,True,False,False,False
5,WLS,wisconsin longitudinal study,wisconsin longitudinal study|WLS,1,True,1.0,NaN,NaN,wisconsin,1,no_match,wls,False,wisconsin longitudinal study,174,education longitudinal study,True,False,False,False
6,PIRLS,progress in international reading literacy study,progress in international reading literacy stu...,1,True,15.0,NaN,NaN,progress in international reading literacy,5,no_match,pirls,False,progress in international reading literacy study,175,no_match,True,False,False,False
7,IPEDS,integrated postsecondary education data system,integrated postsecondary education data system...,2,True,18.0,NaN,NaN,integrated postsecondary system,3,no_match,ipeds,False,integrated postsecondary education data system,179,no_match,False,True,False,True
8,NSFH,national survey of families and households,national survey of families and households|NSFH,1,True,1.0,NaN,NaN,families and households,3,no_match,nsfh,False,national survey of families and households,181,no_match,True,False,False,False
9,IPUMS,integrated public use microdata series,integrated public use microdata series|IPUMS,1,True,1.0,NaN,NaN,integrated public use microdata series,5,no_match,ipums,False,integrated public use microdata series,182,no_match,False,False,False,False


In [32]:
df_traintest['det_acronym'] = df_traintest['clean_text'].progress_apply(lambda x: 
    detect_acronym(x, df_label_acronym_ver1_2['cleaned_label'], 
                   df_label_acronym_ver1_2['acronym_clean'], df_label_acronym_ver1_2['target'])
)
df_traintest.head()

100%|██████████| 100/100 [00:00<00:00, 202.00it/s]


,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,clean_text,det_acronym
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...,this study used data from the national educati...,[]
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...,dropping out of high school is not necessarily...,[]
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,...",stress satisfactory outcomes for all youth inc...,[]
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...,federal reserve bank of richmond s1 accounting...,[]
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...,this article investigates an important factor ...,[210]


In [33]:
tmp = df_traintest['det_acronym'].values
tmp = np.concatenate(tmp)

df_label_acronym_ver1_2['count'] = df_label_acronym_ver1_2['target'].apply(lambda x: np.sum(tmp==x))
df_label_acronym_ver1_2 = df_label_acronym_ver1_2.sort_values('count', ascending=False)
df_label_acronym_ver1_2

,acronym,base,base_acronym,count_BA,test,count_BA_train,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym,cleaned_label,target,match_train,keyword3,keyword4,black,white,count
35,DUMMY,dummy,dummy|DUMMY,1,True,NaN,NaN,NaN,dummy,1,no_match,dummy,False,dummy,210,no_match,False,False,False,False,14
2,NLSY,national longitudinal survey of youth,national longitudinal survey of youth|NLSY,2,True,4.0,NaN,NaN,youth,1,no_match,nlsy,False,national longitudinal survey of youth,170,no_match,True,False,False,False,4
4,CPS,current population survey,current population survey|CPS,2,True,13.0,NaN,NaN,current population,2,no_match,cps,False,current population survey,173,no_match,True,False,False,False,4
52,PSID,panel study of income dynamics,panel study of income dynamics|PSID,4,NaN,NaN,True,no_match,income dynamics,2,no_match,psid,False,panel study of income dynamics,235,no_match,True,False,False,False,3
27,NHIS,national health interview survey,national health interview survey|NHIS,1,True,2.0,NaN,NaN,health interview,2,no_match,nhis,False,national health interview survey,202,no_match,True,False,False,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,CPS,current population study,current population study|CPS,1,NaN,NaN,True,no_match,current population,2,no_match,cps,False,current population study,302,no_match,True,False,False,False,0
100,LISS,longitudinal internet studies for the social s...,longitudinal internet studies for the social s...,1,NaN,NaN,True,no_match,internet for social sciences,4,no_match,liss,False,longitudinal internet studies for the social s...,303,no_match,True,False,False,False,0
101,EDCCS,eye disease case control study,eye disease case control study|EDCCS,1,NaN,NaN,True,no_match,eye disease case control,4,no_match,edccs,False,eye disease case control study,304,no_match,True,False,False,False,0
102,EHLS,epidemiology of hearing loss study,epidemiology of hearing loss study|EHLS,1,NaN,NaN,True,no_match,epidemiology hearing loss,3,no_match,ehls,False,epidemiology of hearing loss study,305,no_match,True,False,False,False,0


In [34]:
if len(df_test)==4:
    TH_COUNT = 0 # 1 is best for private
else:
    TH_COUNT = 1
idx_tmp =  (df_label_acronym_ver1_2['count']>=TH_COUNT)
df_label_acronym_ver1_black =ri(df_label_acronym_ver1_2[idx_tmp])
idx_tmp =  (df_label_acronym_ver1_2['count']>=TH_COUNT)
idx_tmp = idx_tmp & (df_label_acronym_ver1_2['black']==False)
df_label_acronym_ver1_3 =ri(df_label_acronym_ver1_2[idx_tmp])
print(df_label_acronym_ver1.shape)
print(df_label_acronym_ver1_2.shape)
print(df_label_acronym_ver1_black.shape)
print(df_label_acronym_ver1_3.shape)
df_label_acronym_ver1_3.head(30)

(326, 20)
(253, 21)
(253, 21)
(235, 21)


,acronym,base,base_acronym,count_BA,test,count_BA_train,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym,cleaned_label,target,match_train,keyword3,keyword4,black,white,count
0,DUMMY,dummy,dummy|DUMMY,1,True,NaN,NaN,NaN,dummy,1,no_match,dummy,False,dummy,210,no_match,False,False,False,False,14
1,NLSY,national longitudinal survey of youth,national longitudinal survey of youth|NLSY,2,True,4.0,NaN,NaN,youth,1,no_match,nlsy,False,national longitudinal survey of youth,170,no_match,True,False,False,False,4
2,CPS,current population survey,current population survey|CPS,2,True,13.0,NaN,NaN,current population,2,no_match,cps,False,current population survey,173,no_match,True,False,False,False,4
3,PSID,panel study of income dynamics,panel study of income dynamics|PSID,4,NaN,NaN,True,no_match,income dynamics,2,no_match,psid,False,panel study of income dynamics,235,no_match,True,False,False,False,3
4,NHIS,national health interview survey,national health interview survey|NHIS,1,True,2.0,NaN,NaN,health interview,2,no_match,nhis,False,national health interview survey,202,no_match,True,False,False,False,3
5,SASS,schools and staffing survey,schools and staffing survey|SASS,15,NaN,NaN,True,no_match,schools and staffing,3,no_match,sass,False,schools and staffing survey,216,no_match,True,False,False,False,2
6,HSES,high school effectiveness study,high school effectiveness study|HSES,1,True,1.0,NaN,NaN,high school effectiveness,3,no_match,hses,False,high school effectiveness study,185,high school longitudinal study,True,False,False,False,2
7,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,1,True,1.0,NaN,NaN,high school and beyond dataset,5,no_match,hsb,False,high school and beyond longitudinal dataset,167,no_match,True,False,False,False,1
8,QED,quality education data,quality education data|QED,1,NaN,NaN,True,no_match,quality,1,no_match,qed,False,quality education data,215,no_match,False,False,False,False,1
9,NPSAS,national postsecondary student aid study,national postsecondary student aid study|NPSAS,5,NaN,NaN,True,no_match,postsecondary student aid,3,no_match,npsas,False,national postsecondary student aid study,233,no_match,True,False,False,False,1


# Detect Acronym ver1 self

In [35]:
TH_COUNT = 1 # 1 is best for private
df_label_acronym_ver1_self = df_label_acronym_ver1_3.drop('target', axis=1)
df_tmp = ri(df_label_acronym_ver1_self[df_label_acronym_ver1_self['acronym_clean'].duplicated()==False])
df_tmp['target'] = np.arange(len(df_tmp)) + df_label_acronym_ver1_3['target'].max()+1
df_label_acronym_ver1_self = pd.merge(
    df_label_acronym_ver1_self, df_tmp[['acronym_clean', 'target']], on='acronym_clean', how='left')
print(df_label_acronym_ver1.shape)
print(df_label_acronym_ver1_self.shape)
print(df_label_acronym_ver1_self[df_label_acronym_ver1_self['cleaned_label'].duplicated()==False].shape)
df_label_acronym_ver1_self.head(30)

(326, 20)
(235, 21)
(235, 21)


,acronym,base,base_acronym,count_BA,test,count_BA_train,train,match,label-keywords,len_label-keywords,match_train-keywords,acronym_clean,match_train_acronym,cleaned_label,match_train,keyword3,keyword4,black,white,count,target
0,DUMMY,dummy,dummy|DUMMY,1,True,NaN,NaN,NaN,dummy,1,no_match,dummy,False,dummy,no_match,False,False,False,False,14,489
1,NLSY,national longitudinal survey of youth,national longitudinal survey of youth|NLSY,2,True,4.0,NaN,NaN,youth,1,no_match,nlsy,False,national longitudinal survey of youth,no_match,True,False,False,False,4,490
2,CPS,current population survey,current population survey|CPS,2,True,13.0,NaN,NaN,current population,2,no_match,cps,False,current population survey,no_match,True,False,False,False,4,491
3,PSID,panel study of income dynamics,panel study of income dynamics|PSID,4,NaN,NaN,True,no_match,income dynamics,2,no_match,psid,False,panel study of income dynamics,no_match,True,False,False,False,3,492
4,NHIS,national health interview survey,national health interview survey|NHIS,1,True,2.0,NaN,NaN,health interview,2,no_match,nhis,False,national health interview survey,no_match,True,False,False,False,3,493
5,SASS,schools and staffing survey,schools and staffing survey|SASS,15,NaN,NaN,True,no_match,schools and staffing,3,no_match,sass,False,schools and staffing survey,no_match,True,False,False,False,2,494
6,HSES,high school effectiveness study,high school effectiveness study|HSES,1,True,1.0,NaN,NaN,high school effectiveness,3,no_match,hses,False,high school effectiveness study,high school longitudinal study,True,False,False,False,2,495
7,HSB,high school and beyond longitudinal dataset,high school and beyond longitudinal dataset|HSB,1,True,1.0,NaN,NaN,high school and beyond dataset,5,no_match,hsb,False,high school and beyond longitudinal dataset,no_match,True,False,False,False,1,496
8,QED,quality education data,quality education data|QED,1,NaN,NaN,True,no_match,quality,1,no_match,qed,False,quality education data,no_match,False,False,False,False,1,497
9,NPSAS,national postsecondary student aid study,national postsecondary student aid study|NPSAS,5,NaN,NaN,True,no_match,postsecondary student aid,3,no_match,npsas,False,national postsecondary student aid study,no_match,True,False,False,False,1,498


# Govt

In [36]:
df_govt = pd.read_csv("../input/coleridge-ext/datasets_govt.csv")
print(df_govt.shape)
df_govt.head()

(103220, 1)


,title
0,department of agriculture congressional logs f...
1,department of agriculture enterprise data inve...
2,department of agriculture secretary's calendar...
3,realized cost savings and avoidance
4,usda active purchase card holders


In [37]:
df_govt['keyword'] = df_govt['title'].progress_apply(lambda x: detect_keywords(x, keywords))
idx_tmp = df_govt['keyword']
df_govt2 = ri(df_govt[df_govt['keyword']])
df_govt2['cleaned_label'] = df_govt2['title'].progress_apply(clean_text)
df_govt2['match_train'] = df_govt2['cleaned_label'].progress_apply(
    get_match, ref_labels=df_label_train['cleaned_label_my'])

idx_tmp = df_govt2['keyword']
print(df_govt2[idx_tmp].shape)
df_govt2[idx_tmp].head()

100%|██████████| 42177/42177 [00:30<00:00, 1375.30it/s]

(42177, 4)


,title,keyword,cleaned_label,match_train
0,department of agriculture enterprise data inve...,True,department of agriculture enterprise data inve...,no_match
1,usda help desk support data asset,True,usda help desk support data asset,no_match
2,agricultural baseline database,True,agricultural baseline database,no_match
3,agricultural exchange rate data set,True,agricultural exchange rate data set,no_match
4,aquaculture data,True,aquaculture data,no_match


In [38]:
df_govt2['len'] = df_govt2['cleaned_label'].progress_apply(lambda x: len(x.split()))
idx_tmp = df_govt2['match_train']=='no_match'
idx_tmp = df_govt2['len']>=3
df_govt3 = ri(df_govt2[idx_tmp])
print(df_govt2[idx_tmp].shape)
df_govt2[idx_tmp].head()

100%|██████████| 42177/42177 [00:00<00:00, 395977.09it/s]

(42108, 5)


,title,keyword,cleaned_label,match_train,len
0,department of agriculture enterprise data inve...,True,department of agriculture enterprise data inve...,no_match,6
1,usda help desk support data asset,True,usda help desk support data asset,no_match,6
2,agricultural baseline database,True,agricultural baseline database,no_match,3
3,agricultural exchange rate data set,True,agricultural exchange rate data set,no_match,5
5,china agricultural and economic data,True,china agricultural and economic data,no_match,5


In [39]:
df_govt3['match_ver1'] = df_govt2['cleaned_label'].progress_apply(
    get_match, ref_labels=df_label_acronym_ver1_3['cleaned_label'])
idx_tmp = df_govt3['match_ver1']=='no_match'
idx_tmp = idx_tmp & (df_govt3['match_train']=='no_match')
df_govt4 = ri(df_govt3[idx_tmp])
print(df_govt4.shape)
df_govt4.head()

100%|██████████| 42177/42177 [00:37<00:00, 1116.96it/s]

(41930, 6)


,title,keyword,cleaned_label,match_train,len,match_ver1
0,department of agriculture enterprise data inve...,True,department of agriculture enterprise data inve...,no_match,6,no_match
1,usda help desk support data asset,True,usda help desk support data asset,no_match,6,no_match
2,agricultural baseline database,True,agricultural baseline database,no_match,3,no_match
3,agricultural exchange rate data set,True,agricultural exchange rate data set,no_match,5,no_match
4,china agricultural and economic data,True,china agricultural and economic data,no_match,5,no_match


In [40]:
df_govt4['target'] = np.arange(len(df_govt4)) + df_label_acronym_ver1_self['target'].max()+1
df_govt4.head()

,title,keyword,cleaned_label,match_train,len,match_ver1,target
0,department of agriculture enterprise data inve...,True,department of agriculture enterprise data inve...,no_match,6,no_match,696
1,usda help desk support data asset,True,usda help desk support data asset,no_match,6,no_match,697
2,agricultural baseline database,True,agricultural baseline database,no_match,3,no_match,698
3,agricultural exchange rate data set,True,agricultural exchange rate data set,no_match,5,no_match,699
4,china agricultural and economic data,True,china agricultural and economic data,no_match,5,no_match,700


In [41]:
idx_tmp = df_govt4['len']<=10
df_govt5 = df_govt4[idx_tmp]
df_govt5 = ri(df_govt5.sort_values('len'))
print(df_govt5.shape)
df_govt5.head()

(20353, 7)


,title,keyword,cleaned_label,match_train,len,match_ver1,target
0,cdrh inspections database,True,cdrh inspections database,no_match,3,no_match,30548
1,electrical metering data -,True,electrical metering data,no_match,3,no_match,40030
2,alms surveys 2011,True,alms surveys 2011,no_match,3,no_match,31235
3,everglades isotope data,True,everglades isotope data,no_match,3,no_match,38710
4,alms surveys 2010,True,alms surveys 2010,no_match,3,no_match,31237


In [42]:
def func(x, ref):
    index_tmp = x['index']
    if index_tmp==0:
        return 'no_match'
    ref = ref[:index_tmp-1]
    for item in ref:
        if jaccard(x['cleaned_label'], item)>=0.5:
            return item
    return 'no_match'
ref_list = df_govt5['cleaned_label'].tolist()
df_govt5['index'] = np.arange(len(df_govt5))
if len(df_test)==4:
    df_govt5 = df_govt5.iloc[:1000]
df_govt5['match_in_self'] = df_govt5.progress_apply(lambda x: func(x, ref_list), axis=1)

100%|██████████| 1000/1000 [00:01<00:00, 883.55it/s]


In [43]:
idx_tmp = df_govt5['match_in_self']=='no_match'
df_govt6 = ri(df_govt5[idx_tmp])
print(df_govt6.shape)
df_govt6.head()

(155, 9)


,title,keyword,cleaned_label,match_train,len,match_ver1,target,index,match_in_self
0,cdrh inspections database,True,cdrh inspections database,no_match,3,no_match,30548,0,no_match
1,electrical metering data -,True,electrical metering data,no_match,3,no_match,40030,1,no_match
2,alms surveys 2011,True,alms surveys 2011,no_match,3,no_match,31235,2,no_match
3,everglades isotope data,True,everglades isotope data,no_match,3,no_match,38710,3,no_match
4,vmat request database -,True,vmat request database,no_match,3,no_match,40828,5,no_match


In [44]:
df_traintest['det_govt'] = df_traintest['clean_text'].progress_apply(lambda x: 
    detect_label(x.lower(), ref_label=df_govt6['cleaned_label'].values, 
                 ref_target=df_govt6['target'].values))

100%|██████████| 100/100 [00:00<00:00, 319.18it/s]


In [45]:
tmp = df_traintest['det_govt'].values
tmp = np.concatenate(tmp)

df_govt6['count'] = df_govt6['target'].apply(lambda x: np.sum(tmp==x))
df_govt6 = ri(df_govt6.sort_values('count', ascending=False))
df_govt6

,title,keyword,cleaned_label,match_train,len,match_ver1,target,index,match_in_self,count
0,current population survey,True,current population survey,no_match,3,no_match,29842,8,no_match,9
1,cdrh inspections database,True,cdrh inspections database,no_match,3,no_match,30548,0,no_match,0
2,about healthdata.gov,True,about healthdata gov,no_match,3,no_match,30777,125,no_match,0
3,black skimmer survey,True,black skimmer survey,no_match,3,no_match,33805,117,no_match,0
4,global land survey,True,global land survey,no_match,3,no_match,37363,118,no_match,0
...,...,...,...,...,...,...,...,...,...,...
150,enterprise data inventory,True,enterprise data inventory,no_match,3,no_match,38959,62,no_match,0
151,lobster logbook data,True,lobster logbook data,no_match,3,no_match,29352,63,no_match,0
152,sap project data,True,sap project data,no_match,3,no_match,39357,64,no_match,0
153,wildlife utilization survey,True,wildlife utilization survey,no_match,3,no_match,32324,65,no_match,0


In [46]:
TH_COUNT = 2
idx_tmp = df_govt6['count']>=TH_COUNT
idx_tmp = idx_tmp & (df_govt6['match_train']=='no_match')
df_govt7 = ri(df_govt6[idx_tmp])
print(df_govt7.shape)
df_govt7.head(30)

(1, 10)


,title,keyword,cleaned_label,match_train,len,match_ver1,target,index,match_in_self,count
0,current population survey,True,current population survey,no_match,3,no_match,29842,8,no_match,9


# NER
det1, trainの表記ゆれ検出  
trainはdataset名でグルーピングする
els, nelsは検出しない。

In [47]:
tokenizer = pickle_load("../input/coleridge-ext/tokenizer_210613_01.pkl")

In [48]:
df_label_ref = df_label_acronym_ver1_3[['cleaned_label', 'target']].copy()
df_tmp = df_label_train_ref[['cleaned_label_my', 'target_group']]
df_tmp.columns = ['cleaned_label', 'target']
df_label_ref = ri(pd.concat([df_label_ref, df_tmp], axis=0))
print(df_label_ref.shape)
df_label_ref.head()

(413, 2)


,cleaned_label,target
0,dummy,210
1,national longitudinal survey of youth,170
2,current population survey,173
3,panel study of income dynamics,235
4,national health interview survey,202


In [49]:
df_test_reduced['det_ref'] = df_test_reduced['clean_text'].progress_apply(lambda x: 
    detect_label(x, df_label_ref['cleaned_label'], df_label_ref['target'].values)
)
df_test_reduced['len_det'] = df_test_reduced['det_ref'].apply(len)
print(df_test_reduced[df_test_reduced['len_det']>0].shape) # 340,6
df_test_reduced[df_test_reduced['len_det']>0].head(30)

100%|██████████| 1000/1000 [00:08<00:00, 118.53it/s]


(891, 6)


,Id,text,clean_text,det_acronym,det_ref,len_det
4,c754dec7-c5a3-4337-9892-c02158475064,This article investigates an important factor ...,this article investigates an important factor ...,"[[NELS, national education longitudinal study]]",[210],1
6,f35fdfa6-a3f9-4ef3-a858-c8e2efcd2349,Despite the striking reversal of the gender ga...,despite the striking reversal of the gender ga...,[],[185],1
7,1b21f60a-4022-4b19-95ce-6fd7157d4aa9,"In this article, the authors report the result...",in this article the authors report the results...,[],[233],1
8,e5a4c6af-0854-4f7e-b088-06f03c18a01a,Abstract Past research has shown that locus of...,abstract past research has shown that locus of...,[],"[210, 4]",2
10,e27b58cf-5453-4493-8ec4-f7cd892e8624,The large and persistent achievement gaps sepa...,the large and persistent achievement gaps sepa...,[],[210],1
11,630de001-fa5b-4973-ba3f-52f6f1f4dc2d,Reflecting the value placed on postsecondary e...,reflecting the value placed on postsecondary e...,[],"[21, 24, 21, 24, 21]",5
12,d5ff0ecd-ccf5-4f8a-a09c-182376eed4d0,"The black-white gap in achievement, as measure...",the black white gap in achievement as measured...,[],[210],1
13,e977a870-0067-4221-8423-5ff349510222,This study examines two important and related ...,this study examines two important and related ...,[],"[210, 185]",2
14,7a2d20d9-fe83-4d24-b4ce-992f92f21bd2,ABSTRACT\nUsing a dataset which allows student...,abstract using a dataset which allows students...,[],[210],1
15,65731d69-af28-47af-8d29-a72b4a430e37,ABSTRACT: This paper analyses the impact of te...,abstract this paper analyses the impact of tea...,None,[210],1


In [50]:
def get_include(x, ref):
    try:
        for item in ref:
            if item in x:
                return item
        return 'no_include'
    except:
        return 'no_include'
    
def get_include2(x, ref):
    try:
        for item in ref:
            if x in item:
                return item
        return 'no_include'
    except:
        return 'no_include'
    
nels_els_list = [
    'national education longitudinal study',
    'education longitudinal study',
]

In [51]:
%%time
i = 41
BATCH_SIZE = 64
def get_ner_pred(x):
    try:
        text = x['text']
        sentence_list = chunk_text(text, keywords=keywords) # keyword含む文のみのchunkを作成
        det_ref = x['det_ref']
        labels_det_ref = []
        for target in det_ref:
            labels_det_ref.append(df_label_ref['cleaned_label'][df_label_ref['target']==target].values[0])
        labels_det_ref = np.unique(labels_det_ref).tolist()

        num_batch = int(np.ceil(len(sentence_list)/BATCH_SIZE))
        naun_chunks = []
        for j in range(num_batch):
            batch = sentence_list[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
            for k, sent in enumerate(batch):
                naun_chunks_k = list(finetuned_ner_nlp(sent).ents) # sentenceで区切ってバッチ化して処理される
                naun_chunks += naun_chunks_k
        naun_chunks = [str(chunk) for chunk in naun_chunks]
        naun_chunks2 = np.unique(naun_chunks)
        df_chunks = pd.DataFrame(naun_chunks2, columns=['chunk'])
        df_chunks['chunk-the'] = df_chunks['chunk'].apply(delete_the)
        df_chunks['cleaned_label'] = df_chunks['chunk-the'].apply(clean_text)
        df_chunks['len'] = df_chunks['cleaned_label'].apply(lambda x: len(x.split()))
        df_chunks['det_already'] = df_chunks['cleaned_label'].apply(lambda x: x in labels_det_ref)
        df_chunks['match_ref'] = df_chunks['cleaned_label'].apply(
            lambda x: get_match(x, ref_labels=df_label_ref['cleaned_label'], threshold=0.5)
        )
        df_chunks['match_nels_els'] = df_chunks['cleaned_label'].apply(
            lambda x: get_match(x, ref_labels=nels_els_list, threshold=0.5)
        )
        df_chunks['include_ref'] = df_chunks['cleaned_label'].apply(
            lambda x: get_include(x, ref=df_label_ref['cleaned_label'])
        )
        
        df_chunks['include_ref2'] = df_chunks['cleaned_label'].apply(
            lambda x: get_include2(x, ref=df_label_ref['cleaned_label'])
        )
        idx_tmp = df_chunks['cleaned_label'].duplicated()==False
        idx_tmp = idx_tmp & (df_chunks['len']>=3)
        idx_tmp = idx_tmp & (df_chunks['det_already']==False)
        idx_tmp = idx_tmp & (df_chunks['match_ref']!='no_match')
        idx_tmp = idx_tmp & (df_chunks['match_nels_els']=='no_match')
        idx_tmp = idx_tmp & (df_chunks['include_ref']=='no_include')
        idx_tmp = idx_tmp & (df_chunks['include_ref2']=='no_include')
        df_chunks2 = ri(df_chunks[idx_tmp])

        pred_ner = "|".join(df_chunks2['cleaned_label'].values.tolist())
#         pred_ner = 'A'
        return pred_ner
    except:
        return ''

df_test_reduced['pred_ner'] = ""
df_test_reduced['pred_ner'].iloc[:20] = df_test_reduced.iloc[:20].progress_apply(get_ner_pred, axis=1)
df_test_reduced.iloc[10:20]

100%|██████████| 20/20 [00:19<00:00,  1.05it/s]

CPU times: user 18 s, sys: 175 ms, total: 18.2 s
Wall time: 19.1 s



/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Id,text,clean_text,det_acronym,det_ref,len_det,pred_ner
10,e27b58cf-5453-4493-8ec4-f7cd892e8624,The large and persistent achievement gaps sepa...,the large and persistent achievement gaps sepa...,[],[210],1,joint center for political studies
11,630de001-fa5b-4973-ba3f-52f6f1f4dc2d,Reflecting the value placed on postsecondary e...,reflecting the value placed on postsecondary e...,[],"[21, 24, 21, 24, 21]",5,
12,d5ff0ecd-ccf5-4f8a-a09c-182376eed4d0,"The black-white gap in achievement, as measure...",the black white gap in achievement as measured...,[],[210],1,
13,e977a870-0067-4221-8423-5ff349510222,This study examines two important and related ...,this study examines two important and related ...,[],"[210, 185]",2,
14,7a2d20d9-fe83-4d24-b4ce-992f92f21bd2,ABSTRACT\nUsing a dataset which allows student...,abstract using a dataset which allows students...,[],[210],1,national study of postgraduate faculty|social ...
15,65731d69-af28-47af-8d29-a72b4a430e37,ABSTRACT: This paper analyses the impact of te...,abstract this paper analyses the impact of tea...,None,[210],1,
16,a8c7306f-908c-4d44-92b1-2afb2e066808,Abstract: Participation in structured out-of-s...,abstract participation in structured out of sc...,[],"[210, 235]",2,
17,2f9d8444-9034-4ecb-a836-b1c473b9c457,Jump down to document Visit RAND at www.rand.o...,jump down to document visit rand at www rand o...,None,[],0,national assessment of educational progress|na...
18,f5cffcf3-bcc6-4ab0-8203-851a17df96bf,PHOTO CREDIT: PHOTOS.COM EDUCATIONFORUM C once...,photo credit photos com educationforum c oncer...,[],[],0,
19,a3c998b6-ea9d-4542-9267-68f718247ff5,dents thus may determine prior to enrolling th...,dents thus may determine prior to enrolling th...,[],"[173, 21]",2,


In [52]:
"""
ver3
10 joint center for political studies
14 national study of postgraduate faculty

"""
df_test_reduced[df_test_reduced['pred_ner']!='']

,Id,text,clean_text,det_acronym,det_ref,len_det,pred_ner
7,1b21f60a-4022-4b19-95ce-6fd7157d4aa9,"In this article, the authors report the result...",in this article the authors report the results...,[],[233],1,national assessment of vocational education
9,4e18b1c7-e195-46e2-af0f-78757259f0c6,sible in summary format. An example would be a...,sible in summary format an example would be a ...,[],[],0,national assessment of educational progress
10,e27b58cf-5453-4493-8ec4-f7cd892e8624,The large and persistent achievement gaps sepa...,the large and persistent achievement gaps sepa...,[],[210],1,joint center for political studies
14,7a2d20d9-fe83-4d24-b4ce-992f92f21bd2,ABSTRACT\nUsing a dataset which allows student...,abstract using a dataset which allows students...,[],[210],1,national study of postgraduate faculty|social ...
17,2f9d8444-9034-4ecb-a836-b1c473b9c457,Jump down to document Visit RAND at www.rand.o...,jump down to document visit rand at www rand o...,None,[],0,national assessment of educational progress|na...


In [53]:
df_label_train_ref['cleaned_label'][df_label_train_ref['target_group']==3]

4          baltimore longitudinal study of aging
10    baltimore longitudinal study of aging blsa
11    baltimore longitudinal study of aging blsa
Name: cleaned_label, dtype: object

In [54]:
label_tmp = 'baltimore longitudinal study on aging'
for i in range(len(df_label_ref)):
    label_i = df_label_ref['cleaned_label'][i]
    if jaccard(label_i, label_tmp)>=0.5:
        print(label_i)

canadian longitudinal study on aging
baltimore longitudinal study of aging
baltimore longitudinal study of aging blsa
baltimore longitudinal study of aging blsa


# Test prediction

In [55]:
def pred_dup(x):
    df_tmp = df_train_reduced2[df_train_reduced2['text']==x]
    if len(df_tmp)>0:
        label_list = df_tmp['targets'].values[0]
    else:
        label_list = np.zeros(0, np.int64)
    return label_list

df_test['det_dup'] = df_test['text'].apply(lambda x: pred_dup(x))
df_test.head()

,Id,PredictionString,text,clean_text,dup_id,dup,det_dup
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...,2100032a-7c33-4bff-97ef-690822c43466,False,"[0, 1]"
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...,this report describes how the education system...,2f392438-e215-4169-bebf-21ac4ff253e1,True,[2]
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...,3f316b38-1a24-45a9-8d8c-4e05a42257c6,True,"[14, 126]"
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...,a significant body of research has been conduc...,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,True,[9]


In [56]:
df_test['det_train'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_label(x.lower(), ref_label=df_label_train['label'].values, 
                 ref_target=df_label_train['target_relabeled'].values))

100%|██████████| 4/4 [00:00<00:00, 150.19it/s]


In [57]:
df_test['det_train_acronym'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_acronym(x, df_label_train_acronym2['base'], 
                   df_label_train_acronym2['acronym_clean'], 
                   df_label_train_acronym2['target'], th=[1,2])
)
df_test.head()

100%|██████████| 4/4 [00:00<00:00, 276.71it/s]


,Id,PredictionString,text,clean_text,dup_id,dup,det_dup,det_train,det_train_acronym
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...,2100032a-7c33-4bff-97ef-690822c43466,False,"[0, 1]","[0, 1]",[]
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...,this report describes how the education system...,2f392438-e215-4169-bebf-21ac4ff253e1,True,[2],"[2, 19, 45]",[149]
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...,3f316b38-1a24-45a9-8d8c-4e05a42257c6,True,"[14, 126]","[14, 126]","[131, 131, 131]"
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...,a significant body of research has been conduc...,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,True,[9],[9],"[155, 155]"


In [58]:
df_test['det_govt'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_label(x.lower(), ref_label=df_govt7['cleaned_label'].values, 
                 ref_target=df_govt7['target'].values))

100%|██████████| 4/4 [00:00<00:00, 2029.91it/s]


In [59]:
df_test['det_acronym'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_acronym(x, df_label_acronym_ver1_3['cleaned_label'], 
                   df_label_acronym_ver1_3['acronym_clean'], df_label_acronym_ver1_3['target'], th=[1,0])
)
df_test.head()

100%|██████████| 4/4 [00:00<00:00, 98.94it/s]


,Id,PredictionString,text,clean_text,dup_id,dup,det_dup,det_train,det_train_acronym,det_govt,det_acronym
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...,2100032a-7c33-4bff-97ef-690822c43466,False,"[0, 1]","[0, 1]",[],[],"[186, 204, 207]"
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...,this report describes how the education system...,2f392438-e215-4169-bebf-21ac4ff253e1,True,[2],"[2, 19, 45]",[149],[29842],"[173, 216, 179, 175]"
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...,3f316b38-1a24-45a9-8d8c-4e05a42257c6,True,"[14, 126]","[14, 126]","[131, 131, 131]",[],[229]
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...,a significant body of research has been conduc...,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,True,[9],[9],"[155, 155]",[],[206]


In [60]:
df_test['det_acronym_self'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_acronym(x, df_label_acronym_ver1_self['cleaned_label'], 
                   df_label_acronym_ver1_self['acronym_clean'], df_label_acronym_ver1_self['target'], th=[1,1])
)
df_test.head()

100%|██████████| 4/4 [00:00<00:00, 99.23it/s]


,Id,PredictionString,text,clean_text,dup_id,dup,det_dup,det_train,det_train_acronym,det_govt,det_acronym,det_acronym_self
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...,2100032a-7c33-4bff-97ef-690822c43466,False,"[0, 1]","[0, 1]",[],[],"[186, 204, 207]","[633, 641, 642]"
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...,this report describes how the education system...,2f392438-e215-4169-bebf-21ac4ff253e1,True,[2],"[2, 19, 45]",[149],[29842],"[173, 216, 179, 175]","[491, 494, 503, 515]"
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...,3f316b38-1a24-45a9-8d8c-4e05a42257c6,True,"[14, 126]","[14, 126]","[131, 131, 131]",[],[229],[610]
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...,a significant body of research has been conduc...,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,True,[9],[9],"[155, 155]",[],[206],[]


In [61]:
df_test['det_ref'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_label(x, df_label_ref['cleaned_label'], df_label_ref['target'].values)
)

100%|██████████| 4/4 [00:00<00:00, 42.03it/s]


In [62]:
df_test['pred_ner'] = df_test.progress_apply(get_ner_pred, axis=1)
df_test.head()

100%|██████████| 4/4 [00:05<00:00,  1.28s/it]


,Id,PredictionString,text,clean_text,dup_id,dup,det_dup,det_train,det_train_acronym,det_govt,det_acronym,det_acronym_self,det_ref,pred_ner
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...,2100032a-7c33-4bff-97ef-690822c43466,False,"[0, 1]","[0, 1]",[],[],"[186, 204, 207]","[633, 641, 642]","[186, 204, 207, 0, 0, 0]",
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...,this report describes how the education system...,2f392438-e215-4169-bebf-21ac4ff253e1,True,[2],"[2, 19, 45]",[149],[29842],"[173, 216, 179, 175]","[491, 494, 503, 515]","[173, 216, 179, 175, 2, 19, 19]",
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...,3f316b38-1a24-45a9-8d8c-4e05a42257c6,True,"[14, 126]","[14, 126]","[131, 131, 131]",[],[229],[610],"[229, 14, 14, 14]",us geological survey
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...,a significant body of research has been conduc...,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,True,[9],[9],"[155, 155]",[],[206],[],"[206, 9, 9]",


In [63]:
df_test['pred_det'] = df_test.progress_apply(lambda x: 
    np.sort(np.unique(np.concatenate([
        x['det_acronym_self'],
        x['det_acronym'],
        x['det_govt'],
        x['det_dup'],
        x['det_train'],
        x['det_train_acronym'],
    ]))).astype(np.int64), axis=1
)
df_test.head()

100%|██████████| 4/4 [00:00<00:00, 1670.87it/s]


,Id,PredictionString,text,clean_text,dup_id,dup,det_dup,det_train,det_train_acronym,det_govt,det_acronym,det_acronym_self,det_ref,pred_ner,pred_det
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...,2100032a-7c33-4bff-97ef-690822c43466,False,"[0, 1]","[0, 1]",[],[],"[186, 204, 207]","[633, 641, 642]","[186, 204, 207, 0, 0, 0]",,"[0, 1, 186, 204, 207, 633, 641, 642]"
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...,this report describes how the education system...,2f392438-e215-4169-bebf-21ac4ff253e1,True,[2],"[2, 19, 45]",[149],[29842],"[173, 216, 179, 175]","[491, 494, 503, 515]","[173, 216, 179, 175, 2, 19, 19]",,"[2, 19, 45, 149, 173, 175, 179, 216, 491, 494,..."
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...,3f316b38-1a24-45a9-8d8c-4e05a42257c6,True,"[14, 126]","[14, 126]","[131, 131, 131]",[],[229],[610],"[229, 14, 14, 14]",us geological survey,"[14, 126, 131, 229, 610]"
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...,a significant body of research has been conduc...,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,True,[9],[9],"[155, 155]",[],[206],[],"[206, 9, 9]",,"[9, 155, 206]"


In [64]:
df_tmp1 = df_label_acronym_ver1_self[['acronym_clean', 'target']]
df_tmp1.columns = ['label', 'target']
df_tmp2 = df_label_acronym_ver1_3[['cleaned_label', 'target']]
df_tmp2.columns = ['label', 'target']
df_tmp3 = df_govt7[['cleaned_label', 'target']]
df_tmp3.columns = ['label', 'target']
df_tmp4 = df_label_train[['cleaned_label_my', 'target']]
df_tmp4.columns = ['label', 'target']
df_tmp5 = df_label_train_acronym2[['acronym_clean', 'target']]
df_tmp5.columns = ['label', 'target']
df_label_all = pd.concat([df_tmp1, df_tmp2, df_tmp3, df_tmp4, df_tmp5]).reset_index(drop=True)
df_label_all = ri(df_label_all[df_label_all['target'].duplicated()==False])
df_label_all.tail()

,label,target
593,cord 19,158
594,cccsl,159
595,jh crown,160
596,charybdis,161
597,ricord,162


In [65]:
df_test['PredictionString'] = df_test['pred_det'].progress_apply(lambda x: 
    get_label(x, df_label_all['label'].values, df_label_all['target'].values)
)
df_test['PredictionString'] = df_test['PredictionString'] + '|' + df_test['pred_ner']
df_test['PredictionString'] = df_test['PredictionString'].apply(lambda x: x.strip('|'))
df_test.head()

100%|██████████| 4/4 [00:00<00:00, 2006.36it/s]


,Id,PredictionString,text,clean_text,dup_id,dup,det_dup,det_train,det_train_acronym,det_govt,det_acronym,det_acronym_self,det_ref,pred_ner,pred_det
0,2100032a-7c33-4bff-97ef-690822c43466,adni|alzheimer s disease neuroimaging initiati...,Cognitive deficits and reduced educational ach...,cognitive deficits and reduced educational ach...,2100032a-7c33-4bff-97ef-690822c43466,False,"[0, 1]","[0, 1]",[],[],"[186, 204, 207]","[633, 641, 642]","[186, 204, 207, 0, 0, 0]",,"[0, 1, 186, 204, 207, 633, 641, 642]"
1,2f392438-e215-4169-bebf-21ac4ff253e1,common core of data|cps|current population sur...,This report describes how the education system...,this report describes how the education system...,2f392438-e215-4169-bebf-21ac4ff253e1,True,[2],"[2, 19, 45]",[149],[29842],"[173, 216, 179, 175]","[491, 494, 503, 515]","[173, 216, 179, 175, 2, 19, 19]",,"[2, 19, 45, 149, 173, 175, 179, 216, 491, 494,..."
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,national geodetic survey|ngs|noaa storm surge ...,"Cape Hatteras National Seashore (CAHA), locate...",cape hatteras national seashore caha located a...,3f316b38-1a24-45a9-8d8c-4e05a42257c6,True,"[14, 126]","[14, 126]","[131, 131, 131]",[],[229],[610],"[229, 14, 14, 14]",us geological survey,"[14, 126, 131, 229, 610]"
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,national health and nutrition examination surv...,A significant body of research has been conduc...,a significant body of research has been conduc...,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,True,[9],[9],"[155, 155]",[],[206],[],"[206, 9, 9]",,"[9, 155, 206]"


In [66]:
df_sub = df_test[['Id', 'PredictionString']]
df_sub.to_csv("submission.csv", index=None)
df_sub.head()

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,adni|alzheimer s disease neuroimaging initiati...
1,2f392438-e215-4169-bebf-21ac4ff253e1,common core of data|cps|current population sur...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,national geodetic survey|ngs|noaa storm surge ...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,national health and nutrition examination surv...


In [67]:
for i in range(len(df_sub.iloc[:10])):
    print(df_sub['PredictionString'][i])

adni|alzheimer s disease neuroimaging initiative adni|cardiovascular health study|charge|chs|cohorts for heart and aging research in genomic epidemiology|fhs|framingham heart study
common core of data|cps|current population survey|integrated postsecondary education data system|ipeds|nces common core of data|pirls|progress in international reading literacy study|sass|schools and staffing survey|timss|trends in international mathematics and science study
national geodetic survey|ngs|noaa storm surge inundation|sea lake and overland surges from hurricanes|slosh|us geological survey
national health and nutrition examination survey|rucc|rural urban continuum codes


In [68]:
for i in range(len(df_sub.iloc[:10])):
    pred_list = df_sub['PredictionString'][i].split('|')
#     print(pred_list, len(pred_list))
    if pred_list[0]!='':
        for j in range(len(pred_list)):
            print("{:4d}: {}".format(i, pred_list[j]))

   0: adni
   0: alzheimer s disease neuroimaging initiative adni
   0: cardiovascular health study
   0: charge
   0: chs
   0: cohorts for heart and aging research in genomic epidemiology
   0: fhs
   0: framingham heart study
   1: common core of data
   1: cps
   1: current population survey
   1: integrated postsecondary education data system
   1: ipeds
   1: nces common core of data
   1: pirls
   1: progress in international reading literacy study
   1: sass
   1: schools and staffing survey
   1: timss
   1: trends in international mathematics and science study
   2: national geodetic survey
   2: ngs
   2: noaa storm surge inundation
   2: sea lake and overland surges from hurricanes
   2: slosh
   2: us geological survey
   3: national health and nutrition examination survey
   3: rucc
   3: rural urban continuum codes
